
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Professional/main/Includes/images/gold.png" width="60%">
</div>

In [0]:
%run ../Includes/Copy-Datasets

Data catalog: workspace
Schema: bookstore_eng_pro


In [0]:
%sql

CREATE VIEW IF NOT EXISTS countries_stats_vw AS (
  SELECT country, date_trunc("DD", order_timestamp) order_date, count(order_id) orders_count, sum(quantity) books_count
  FROM customers_orders
  GROUP BY country, date_trunc("DD", order_timestamp)
)

In [0]:
%sql
SELECT *
FROM countries_stats_vw
WHERE country = "France"

country,order_date,orders_count,books_count
France,2022-04-06T00:00:00.000Z,2,3
France,2022-04-12T00:00:00.000Z,2,2
France,2022-04-13T00:00:00.000Z,1,2
France,2022-04-16T00:00:00.000Z,3,4
France,2022-04-17T00:00:00.000Z,2,4
France,2022-04-20T00:00:00.000Z,2,2
France,2022-04-21T00:00:00.000Z,1,1
France,2022-04-24T00:00:00.000Z,3,5
France,2022-04-26T00:00:00.000Z,2,2
France,2022-04-19T00:00:00.000Z,2,4


In [0]:
from pyspark.sql import functions as F

query = (spark.readStream
                 .table("books_sales")
                 .withWatermark("order_timestamp", "10 minutes")
                 .groupBy(
                     F.window("order_timestamp", "5 minutes").alias("time"),
                     "author")
                 .agg(
                     F.count("order_id").alias("orders_count"),
                     F.avg("quantity").alias ("avg_quantity"))
              .writeStream
                 .option("checkpointLocation", f"{bookstore.checkpoint_path}/authors_stats")
                 .trigger(availableNow=True)
                 .table("authors_stats")
            )

query.awaitTermination()

In [0]:
%sql
SELECT * FROM authors_stats

time,author,orders_count,avg_quantity
"List(2022-04-23T11:05:00.000Z, 2022-04-23T11:10:00.000Z)",Peter Brass,1,2.0
"List(2022-03-30T12:00:00.000Z, 2022-03-30T12:05:00.000Z)",Peter Brass,1,1.0
"List(2022-04-22T00:10:00.000Z, 2022-04-22T00:15:00.000Z)",Peter Brass,1,2.0
"List(2022-04-28T11:00:00.000Z, 2022-04-28T11:05:00.000Z)",Peter Brass,1,2.0
"List(2021-12-11T18:10:00.000Z, 2021-12-11T18:15:00.000Z)",Peter Brass,1,2.0
"List(2022-05-11T17:05:00.000Z, 2022-05-11T17:10:00.000Z)",Peter Brass,1,1.0
"List(2022-05-03T17:05:00.000Z, 2022-05-03T17:10:00.000Z)",Peter Brass,1,2.0
"List(2021-12-19T15:05:00.000Z, 2021-12-19T15:10:00.000Z)",Addy Osmani,1,2.0
"List(2022-01-04T16:00:00.000Z, 2022-01-04T16:05:00.000Z)",Peter Brass,1,2.0
"List(2022-02-09T00:00:00.000Z, 2022-02-09T00:05:00.000Z)",Rod Stephens,1,2.0
